In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import itertools


from utilities.constants import *
from utilities.functions import *


In [2]:
print("Folder structure in MEASURED_DATA_DIR:")
list_folders_recursively(MEASURED_DATA_DIR)

Folder structure in MEASURED_DATA_DIR:
└── Glass-IV-VC_CC/
  └── CC/
    └── CC/
    └── dev-B1/
    └── dev-B10/
    └── dev-B2/
    └── dev-G2/
  └── VC/
    └── VC/
    └── dev-B1/
    └── dev-B10/
  └── dev-B11-RT/
  └── test-both/
└── Oscilloscope - V3O5/
  └── delay-geometry/
  └── dev-F3/
  └── dev-e11-correct/
  └── dev-e12/
  └── this is e10/


## Glass IV Measurements

In [3]:

# Plot style settings
plt.style.use('seaborn-v0_8-darkgrid')
DEFAULT_FIG_SIZE = (12, 8)
DEFAULT_FONT_SIZE = 12
DEFAULT_LINE_WIDTH = 2
DEFAULT_MARKER_SIZE = 6

# Color settings
COLOR_CYCLE = plt.rcParams['axes.prop_cycle'].by_key()['color']
DEFAULT_ALPHA = 0.7

# Plot formatting
TITLE_FONT_SIZE = 16
LABEL_FONT_SIZE = 14
TICK_FONT_SIZE = 12
LEGEND_FONT_SIZE = 12

# File naming
PLOT_FILE_EXTENSION = '.png'
PLOT_DPI = 300


cc_data = load_data_from_directory(os.path.join(GLASS_IV_DATA_DIR, 'CC'))
vc_data = load_data_from_directory(os.path.join(GLASS_IV_DATA_DIR, 'VC'))
dev_b11_rt_data = load_data_from_directory(os.path.join(GLASS_IV_DATA_DIR, 'dev-B11-RT'))
test_both_data = load_data_from_directory(os.path.join(GLASS_IV_DATA_DIR, 'test-both'))

print(f"Number of files loaded in CC directory: {len(cc_data)}")
print(f"Number of files loaded in VC directory: {len(vc_data)}")
print(f"Number of files loaded in dev-B11-RT directory: {len(dev_b11_rt_data)}")
print(f"Number of files loaded in test-both directory: {len(test_both_data)}")

print("\nSample of first dataframe from each directory:")

folder_name_data = {
    "CC": cc_data,
    "VC": vc_data,
    "dev-B11-RT": dev_b11_rt_data,
    "test-both": test_both_data
}

# Print all columns for each dataframe
for name, data_list in folder_name_data.items():
    if data_list:
        print(f"\n{name} directory first dataframe columns:")
        print(data_list[0].columns)
    else:
        print(f"\n{name} directory: No data loaded")

Number of files loaded in CC directory: 71
Number of files loaded in VC directory: 86
Number of files loaded in dev-B11-RT directory: 1
Number of files loaded in test-both directory: 4

Sample of first dataframe from each directory:

CC directory first dataframe columns:
Index(['Current (A)', 'Volatge (V)', 'Resistance (Ohm)', 'Time (sec)'], dtype='object')

VC directory first dataframe columns:
Index(['Voltage (V)', 'Current (A)', 'Resistance (Ohm)', 'Time (sec)'], dtype='object')

dev-B11-RT directory first dataframe columns:
Index(['Temperature', 'Resistance', 'Time', 'Current', 'V+', 'V-', 'T+', 'T-'], dtype='object')

test-both directory first dataframe columns:
Index(['Current', 'Voltage', 'Resistance', 'Time', 'Temperature'], dtype='object')


In [4]:

def create_combination_plots(data_list, folder_name, output_dir):
    if not data_list:
        print(f"No data available for {folder_name}")
        return
    
    columns = data_list[0].columns
    column_permutations = list(itertools.permutations(columns, 2))
    
    # Convert output_dir to Path if it's a string
    output_dir = Path(output_dir) if isinstance(output_dir, str) else output_dir
    dataset_dir = output_dir / folder_name
    dataset_dir.mkdir(exist_ok=True)
    
    for x_col, y_col in column_permutations:
        plt.figure(figsize=DEFAULT_FIG_SIZE)
        
        for i, df in enumerate(data_list):
            plt.plot(df[x_col], df[y_col], 
                    linewidth=DEFAULT_LINE_WIDTH,
                    alpha=DEFAULT_ALPHA,
                    label=f'File {i+1}')
        
        plt.title(f'{y_col} vs {x_col} - {folder_name}',
                 fontsize=TITLE_FONT_SIZE)
        plt.xlabel(x_col, fontsize=LABEL_FONT_SIZE)
        plt.ylabel(y_col, fontsize=LABEL_FONT_SIZE)
        plt.xticks(fontsize=TICK_FONT_SIZE)
        plt.yticks(fontsize=TICK_FONT_SIZE)
        
        filename = f"{x_col}_vs_{y_col}{PLOT_FILE_EXTENSION}"
        plt.savefig(dataset_dir / filename, dpi=PLOT_DPI, bbox_inches='tight')
        plt.close()

def plot_all_data(dataset_dict, output_dir):
    # Convert output_dir to Path if it's a string
    output_dir = Path(output_dir) if isinstance(output_dir, str) else output_dir
    
    for folder_name, data_list in dataset_dict.items():
        print(f"Creating plots for {folder_name}...")
        create_combination_plots(data_list, folder_name, output_dir)
        print(f"Completed plots for {folder_name}") 

In [5]:
plot_all_data(folder_name_data, f'{IMAGES_DATA_DIR}/glass_iv_plots')

Creating plots for CC...
Completed plots for CC
Creating plots for VC...
Completed plots for VC
Creating plots for dev-B11-RT...
Completed plots for dev-B11-RT
Creating plots for test-both...
Completed plots for test-both
